In [ ]:
--package.path = package.path .. ';/home/kivan/source/deep-learning/semantic_segmentation/torch/?.lua'
--require 'eval_helper'

In [1]:
require 'image';
Plot = require 'itorch.Plot'
paths.dofile('./../eval_helper.lua')

In [5]:
function DrawPrediction(y)
  local h = y:size(1)
  local w = y:size(2)
  local img = torch.ByteTensor(3, h, w):fill(0)
  local cimg = torch.data(img:contiguous())
  local cy = torch.data(y:int():contiguous())
  local s1 = h * w
  for i = 0, h-1 do
    local s2 = i * w
    for j = 0, w-1 do
      --local l = y[{i,j}]
      local l = cy[s2 + j]
      --print(l)
      local color = param_label_colors[l]
      for c = 0, 2 do
        --img[{c,i,j}] = color[c]
        cimg[c*s1 + s2 + j] = color[c+1]
      end
    end
  end
  --image.save(img_save_dir .. img_num, img)
  itorch.image(img)
end

function PlotStats(net, data)
  net:evaluate()
  local loss_sum = 0
  local total_num_labels = 0
  local total_num_correct = 0
  local x, yt, class_weights, filename, depth_img = data:GetNextBatch()
  local num_batches = 0
  local num_classes = param_num_classes
  local confusion_matrix = torch.IntTensor(num_classes, num_classes):fill(0)
  local max_disp = 200
  --local disp_correct_cnt = torch.FloatTensor(max_disp):fill(0)
  --local disp_total_cnt = torch.FloatTensor(max_disp):fill(0)
  local disp_correct_cnt = torch.FloatTensor(num_classes, max_disp):fill(0)
  local disp_total_cnt = torch.FloatTensor(num_classes, max_disp):fill(0)
  while x do
    num_batches = num_batches + 1
    local y = net:forward(x)
    --local E = loss:forward(y, yt)
    --loss_sum = loss_sum + E
    local _, pred = y:max(2)
    pred = pred[1][1]
    table.insert(pred_table, {filename, pred, yt[1]})

    --SaveResult(pred, filename)
    local ignore_mask = yt:ne(0)
    --local num_labels = ignore_mask:sum()
    --total_num_labels = total_num_labels + num_labels
    local error_mat = pred:eq(yt):cmul(ignore_mask)
    --local num_correct = error_mat:sum()
    --total_num_correct = total_num_correct + num_correct

    AggregateStatsWithDepth(pred:int(), yt[1]:int(), confusion_matrix,
                   depth_img, disp_correct_cnt, disp_total_cnt)
    --AggregateConfusionMatrix(pred:int(), yt[1]:int(), confusion_matrix,
    --                         depth_img, disp_correct_cnt, disp_total_cnt)
    --AddToDepthStats(error_mat, depth_img, disp_errors, disp_cnt)
    --xlua.print("\nPixel error = ", 100.0 * num_errors / num_labels)
    --local loss_val = E / yt:ne(0):sum()
    --xlua.print("Loss value = ", loss_val)

    --err = err + loss_helper.ErrorCount(y, labels)
    --dist_err = dist_err + loss_helper.DistanceErrorSum(y, labels)
    --print(y)
    --xlua.print('dist_err = ', dist_err / num_examples)
    --xlua.print('percent_err = ', err / num_examples)
    --xlua.progress(num_batches, data:size())
    x, yt, class_weights, filename, depth_img = data:GetNextBatch()
    collectgarbage()
  end

  --acc_per_disp:cdiv(pixels_per_disp)
  --gnuplot.figure()
  --gnuplot.plot({"accuracy per disp", acc_per_disp})
  --local disp_acc = disp_correct_cnt:cdiv(disp_total_cnt)
  ----xlua.print(disp_acc)
  --gnuplot.figure()
  --gnuplot.plot({'depth accuracy', disp_acc})

  --for i = 1, 3 do
  --  disp_correct_cnt[i]:cdiv(disp_total_cnt[i]:add(1))
  --  --gnuplot.figure(5+i)
  --  --gnuplot.plot({'class_'..i, disp_acc})
  --  table.insert(plot_data, {'class '..i, disp_correct_cnt[i]})
  --end
  
  --xlua.print(confusion_matrix)
  --local avg_pixel_acc2 = total_num_correct / total_num_labels
  local num_correct = confusion_matrix:trace()
  local total_size = confusion_matrix:sum()
  local avg_pixel_acc = num_correct / total_size
  local size_per_class = confusion_matrix:sum(1)[1]
  local class_acc = torch.FloatTensor(size_per_class:size(1))
  xlua.print(string.format("Valid avg pixel accuracy = %.2f %%", avg_pixel_acc * 100.0))
  for i = 1, size_per_class:size(1) do
    class_acc[i] = confusion_matrix[{i,i}] / size_per_class[i] * 100.0
    xlua.print(string.format("\t%s accuracy = %.2f %%", param_label_colors[i][4], class_acc[i]))
  end
  local avg_class_acc = class_acc:mean()
  xlua.print(string.format("Valid avg class accuracy = %.2f %%", avg_class_acc))
    
  local plot_data = {}
  local class_errors_per_disp = disp_total_cnt - disp_correct_cnt
  local class_acc_per_disp = disp_correct_cnt:clone()
  disp_total_cnt:add(1)
  class_acc_per_disp:cdiv(disp_total_cnt)
  disp_total_cnt:csub(1)

  local pixels_per_disp = disp_total_cnt:sum(1)[1]
  --local errors_per_disp = class_errors_per_disp:sum(1)[1]
  --local acc_per_disp = disp_correct_cnt:sum(1)[1]

  local x = torch.linspace(0, max_disp-1, max_disp)
  Plot():line(x, pixels_per_disp, 'red', 'pixels per disp'):legend(true):title('pixels per disp'):draw()
  for i = 1, num_classes do
  --for i = 1, 1 do
    Plot():line(x, class_errors_per_disp[i], 'red', 'errors per disp'):legend(true):title(param_label_colors[i][4]):draw()
    Plot():line(x, class_acc_per_disp[i], 'blue', 'accuracy per disp'):legend(true):title(param_label_colors[i][4]):draw()
  end

  return loss_sum / total_size, avg_pixel_acc, avg_class_acc
end

In [6]:
--local net_dir = '/home/kivan/source/deep-learning/semantic_segmentation/output/results/ThuFeb1811:00:172016/'
local net_dir = '/home/kivan/source/deep-learning/semantic_segmentation/output/nets/16s_SunMar615:58:072016/'
local model_path = net_dir .. '/model_copy.lua'
--local model_path = net_dir .. '/model.lua'
net, loss, train_container, validation_container = paths.dofile(model_path)
net = torch.load(net_dir .. "net.bin")
net:evaluate()
--table1 = {}
--table2 = {}
pred_table = {}

399	


46	


In [7]:
PlotStats(net, validation_container)

Valid avg pixel accuracy = 83.58 %
	sky accuracy = 92.06 %
	building accuracy = 86.88 %
	road accuracy = 85.57 %
	sidewalk accuracy = 84.64 %
	fence accuracy = 46.86 %
	vegetation accuracy = 87.30 %
	pole accuracy = 76.72 %
	car accuracy = 83.93 %
	sign accuracy = 80.53 %
	pedestrian accuracy = 40.82 %
	cyclist accuracy = 17.73 %
Valid avg class accuracy = 71.19 %


0	0.83581076042631	71.185272736983	


In [ ]:
for i = 1, #pred_table do
--for i = 1, 20 do
  --print(pred_table[i][1])
  DrawPrediction(pred_table[i][2])
  DrawPrediction(pred_table[i][3])
end

In [ ]:
print(table1[1][2])
print(table2[1][2])
--DrawPrediction(table1[1][1])
--DrawPrediction(table2[1][1])
local s = 0
for i = 1, #table1 do
  s = s + table1[1][1]:ne(table2[1][1]):sum()
end
print(s)

In [ ]:
table1[1]:ne(table2[1]):sum()

In [ ]:
x1 = torch.randn(40):mul(100)
y1 = torch.randn(40):mul(100)
x2 = torch.randn(40):mul(100)
y2 = torch.randn(40):mul(100)
x3 = torch.randn(40):mul(200)
y3 = torch.randn(40):mul(200)

In [ ]:
-- scatter plots
--plot = Plot():circle(x1, y1, 'red', 'hi'):circle(x2, y2, 'blue', 'bye'):draw()
--plot:circle(x3,y3,'green', 'yolo'):redraw()
--plot:title('Scatter Plot Demo'):redraw()
--plot:xaxis('length'):yaxis('width'):redraw()
--plot:legend(true)
--plot:redraw()
print(x1:dim())
Plot():line(x1,y1,'red','example'):legend(true):title('Line Plot Demo'):draw()
plot = Plot():segment(x1, y1, x1+10,y1+10, 'red','demo'):title('Segment Plot Demo'):draw()
plot = Plot():segment(x1, y1, x1+10,y1+10, 'red','demo'):title('Segment Plot Demo'):draw()
plot = Plot():segment(x1, y1, x1+10,y1+10, 'red','demo'):title('Segment Plot Demo'):draw()

In [ ]:
help(Plot)

In [ ]:
img=image.load('/home/kivan/datasets/KITTI/segmentation/semantic_segmentation/union/torch/1216x352/img/train/rgb/00_000000.png')
print(img:size(2)/8, img:size(3)/8)
img = image.scale(img, img:size(3)/8, img:size(2)/8, 'bicubic')
image.save('/tmp/torch.png', img)
itorch.image(img)

In [ ]:
help(image.save)

In [ ]:
gm = require 'graphicsmagick'

In [ ]:
img1=gm.Image('/home/kivan/datasets/KITTI/segmentation/semantic_segmentation/union/torch/1216x352/img/train/rgb/00_000000.png')
print(img1:size())
img = img1
img:size(152,44)
print(img1:size())
img:save('/tmp/gm.png')